In [898]:
import pandas as pd
import matplotlib as mpl
import platform
import warnings
import numpy as np
import re

In [899]:
warnings.filterwarnings("ignore")

In [900]:
# 한글 깨짐 방지 코드
if platform.system() == 'Windows':
    # Windows의 경우 'Malgun Gothic'을 많이 사용합니다.
    mpl.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':
    # macOS의 경우 'AppleGothic'을 사용하거나, 설치된 한글 폰트를 선택합니다.
    mpl.rcParams['font.family'] = 'AppleGothic'
else:
    # Linux의 경우 'NanumGothic' 등 한글 지원 폰트를 사용할 수 있습니다.
    mpl.rcParams['font.family'] = 'NanumGothic'

In [901]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

### 1 info() 결과
1.1 train info() 결과
- primary key = ID
- 범주형 변수 : 국가, 분야, 투자단계, 인수여부, 상장여부, 기업가치(백억원)
- null값 존재 변수 : 분야, 직원 수, 고객수(백만명), 기업가치(백억원)
- target 변수 : 성공확률
- 모델 돌릴때 null값 처리하고 범주형 변수 처리하고 ID, 성공확률 컬럼 제거
### 
1.2 test info() 결과
- primary key = ID
- null값 존재 컬럼 : 분야, 직원 수, 고객수(백만명), 기업가치(백억원)
- 범주형 변수 : 국가, 분야, 투자단계, 인수여부, 상장여부, 기업가치(백억원)

In [902]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              4376 non-null   object 
 1   설립연도            4376 non-null   int64  
 2   국가              4376 non-null   object 
 3   분야              3519 non-null   object 
 4   투자단계            4376 non-null   object 
 5   직원 수            4202 non-null   float64
 6   인수여부            4376 non-null   object 
 7   상장여부            4376 non-null   object 
 8   고객수(백만명)        3056 non-null   float64
 9   총 투자금(억원)       4376 non-null   float64
 10  연매출(억원)         4376 non-null   float64
 11  SNS 팔로워 수(백만명)  4376 non-null   float64
 12  기업가치(백억원)       3156 non-null   object 
 13  성공확률            4376 non-null   float64
dtypes: float64(6), int64(1), object(7)
memory usage: 478.8+ KB


In [903]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1755 entries, 0 to 1754
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              1755 non-null   object 
 1   설립연도            1755 non-null   int64  
 2   국가              1755 non-null   object 
 3   분야              1401 non-null   object 
 4   투자단계            1755 non-null   object 
 5   직원 수            1679 non-null   float64
 6   인수여부            1755 non-null   object 
 7   상장여부            1755 non-null   object 
 8   고객수(백만명)        1208 non-null   float64
 9   총 투자금(억원)       1755 non-null   float64
 10  연매출(억원)         1755 non-null   float64
 11  SNS 팔로워 수(백만명)  1755 non-null   float64
 12  기업가치(백억원)       1268 non-null   object 
dtypes: float64(5), int64(1), object(7)
memory usage: 178.4+ KB


### 2. 결측값 처리

In [904]:
train['분야'] = train['분야'].fillna('기타')
test['분야'] = test['분야'].fillna('기타')

In [905]:
train['고객수(백만명)'] = train['고객수(백만명)'].fillna(0)
test['고객수(백만명)'] = test['고객수(백만명)'].fillna(0)

In [906]:
from sklearn.impute import KNNImputer

imputer = KNNImputer(n_neighbors=5)
train[['직원 수']] = imputer.fit_transform(train[['직원 수']])
test[['직원 수']] = imputer.fit_transform(test[['직원 수']])

In [907]:
# 기업가치(백억원) 컬럼 null값 처리위해 숫자형 변수로 처리
def parse_기업가치(value):
    if pd.isnull(value):
        return np.nan
    elif '이상' in value:
        # "6000이상" -> 숫자만 추출해서 +500
        num = re.findall(r'\d+', value)
        return float(num[0]) + 500 if num else np.nan
    elif '-' in value:
        # "1500-2500" -> 평균값
        num = re.findall(r'\d+', value)
        if len(num) == 2:
            return (float(num[0]) + float(num[1])) / 2
    return np.nan

train['기업가치(백억원)'] = train['기업가치(백억원)'].apply(parse_기업가치)
test['기업가치(백억원)'] = test['기업가치(백억원)'].apply(parse_기업가치)

In [908]:
# train[기업가치(백억원)] null 값 처리
# 1. 투자단계 + 분야 기준 그룹 평균으로 보간
train['기업가치(백억원)'] = train.groupby(['투자단계', '분야'])['기업가치(백억원)'].transform(
    lambda x: x.fillna(x.mean())
)
# 2. 남은 결측치는 전체 중앙값으로 보간
train['기업가치(백억원)'].fillna(train['기업가치(백억원)'].median(), inplace=True)

# test[기업가치(백억원)] null 값 처리
# 1. 투자단계 + 분야 기준 그룹 평균으로 보간
test['기업가치(백억원)'] = test.groupby(['투자단계', '분야'])['기업가치(백억원)'].transform(
    lambda x: x.fillna(x.mean())
)
# 2. 남은 결측치는 전체 중앙값으로 보간
test['기업가치(백억원)'].fillna(test['기업가치(백억원)'].median(), inplace=True)

### 3. 파생변수 생각해보기

In [909]:
# 설립연차
train['설립연차'] = 2025 - train['설립연도']
test['설립연차'] = 2025 - test['설립연도']

In [910]:
# 기업 성장 속도
train['기업가치_연차비'] = train['기업가치(백억원)'] / train['설립연차']
test['기업가치_연차비'] = test['기업가치(백억원)'] / test['설립연차']

In [911]:
# 투자 효율성
train['매출_투자비'] = train['연매출(억원)'] / train['총 투자금(억원)']
test['매출_투자비'] = test['연매출(억원)'] / test['총 투자금(억원)']

In [912]:
# 브랜드 인지도 대비 실제 고객 전환율
train['팔로워_고객비'] = train['SNS 팔로워 수(백만명)'] / train['고객수(백만명)']
test['팔로워_고객비'] = test['SNS 팔로워 수(백만명)'] / test['고객수(백만명)']
train['팔로워_고객비'].replace([np.inf, -np.inf], np.nan, inplace=True)
train['팔로워_고객비'].fillna(0, inplace=True)
test['팔로워_고객비'].replace([np.inf, -np.inf], np.nan, inplace=True)
test['팔로워_고객비'].fillna(0, inplace=True)

In [913]:
# 이익률 = (연매출 - 총 투자금) / 연매출출
train['이익률'] = (train['연매출(억원)'] - train['총 투자금(억원)']) / train['연매출(억원)']
test['이익률'] = (test['연매출(억원)'] - test['총 투자금(억원)']) / test['연매출(억원)']

In [914]:
# 매출 성장률 = (연매출 - 이전 연매출) / 이전 연매출
train['매출_성장률'] = train['연매출(억원)'].pct_change().fillna(0)
test['매출_성장률'] = test['연매출(억원)'].pct_change().fillna(0)

In [915]:
# 총 자본 비율 = (총 투자금) / (연매출)
train['총_자본_비율'] = train['총 투자금(억원)'] / train['연매출(억원)']
test['총_자본_비율'] = test['총 투자금(억원)'] / test['연매출(억원)']

In [916]:
# 직원당_매출 = 연매출(억원) / 직원 수
train['직원당_매출'] = train['연매출(억원)'] / train['직원 수']
test['직원당_매출'] = test['연매출(억원)'] / test['직원 수']

### 4. 이상치 탐지
- 사분위수 이상치 탐지 결과: 기업가치_연차비, 매출_투자비, 팔로워_고객비 등 파생변수에서만 이상치 발생
- 파생변수 이지만 향후 모델에 사용할 수 있으므로 정제 필요하다고 생각 -> 클리핑 요소 사용
- 클리핑 : 값이 일정 범위를 벗어나면 그 범위의 최대/최소값으로 잘라내는 방법

In [917]:
# IQR 방식
def detect_outliers_iqr(df, columns):
    outlier_info = {}
    
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5 * IQR
        upper = Q3 + 1.5 * IQR

        outliers = df[(df[col] < lower) | (df[col] > upper)]
        outlier_info[col] = {
            'lower_bound': lower,
            'upper_bound': upper,
            'num_outliers': outliers.shape[0]
        }

    return outlier_info

In [918]:
# 수치형 컬럼만 적용
numeric_cols = train.select_dtypes(include=['float64', 'int64']).columns.tolist()
iqr_outliers = detect_outliers_iqr(train, numeric_cols)

# 결과 확인
for col, info in iqr_outliers.items():
    print(f"{col} → 이상치 수: {info['num_outliers']} (범위: {info['lower_bound']:.2f} ~ {info['upper_bound']:.2f})")


설립연도 → 이상치 수: 0 (범위: 1988.00 ~ 2036.00)
직원 수 → 이상치 수: 0 (범위: -2332.88 ~ 7280.12)
고객수(백만명) → 이상치 수: 0 (범위: -93.00 ~ 155.00)
총 투자금(억원) → 이상치 수: 0 (범위: -3233.62 ~ 9793.38)
연매출(억원) → 이상치 수: 0 (범위: -6803.12 ~ 19589.88)
SNS 팔로워 수(백만명) → 이상치 수: 0 (범위: -2.70 ~ 8.02)
기업가치(백억원) → 이상치 수: 0 (범위: -375.00 ~ 8625.00)
성공확률 → 이상치 수: 0 (범위: -0.05 ~ 1.15)
설립연차 → 이상치 수: 0 (범위: -11.00 ~ 37.00)
기업가치_연차비 → 이상치 수: 396 (범위: -345.38 ~ 1121.52)
매출_투자비 → 이상치 수: 512 (범위: -3.10 ~ 7.80)
팔로워_고객비 → 이상치 수: 386 (범위: -0.11 ~ 0.18)
이익률 → 이상치 수: 530 (범위: -1.13 ~ 1.85)
매출_성장률 → 이상치 수: 505 (범위: -2.84 ~ 3.37)
총_자본_비율 → 이상치 수: 530 (범위: -0.85 ~ 2.13)
직원당_매출 → 이상치 수: 500 (범위: -4.49 ~ 10.91)


In [919]:
# 수치형 컬럼만 적용
numeric_cols = test.select_dtypes(include=['float64', 'int64']).columns.tolist()
iqr_outliers = detect_outliers_iqr(test, numeric_cols)

# 결과 확인
for col, info in iqr_outliers.items():
    print(f"{col} → 이상치 수: {info['num_outliers']} (범위: {info['lower_bound']:.2f} ~ {info['upper_bound']:.2f})")

설립연도 → 이상치 수: 0 (범위: 1988.00 ~ 2036.00)
직원 수 → 이상치 수: 0 (범위: -2244.25 ~ 7241.75)
고객수(백만명) → 이상치 수: 0 (범위: -93.00 ~ 155.00)
총 투자금(억원) → 이상치 수: 0 (범위: -3099.50 ~ 9524.50)
연매출(억원) → 이상치 수: 0 (범위: -6550.50 ~ 19489.50)
SNS 팔로워 수(백만명) → 이상치 수: 0 (범위: -2.84 ~ 8.10)
기업가치(백억원) → 이상치 수: 0 (범위: -375.00 ~ 8625.00)
설립연차 → 이상치 수: 0 (범위: -11.00 ~ 37.00)
기업가치_연차비 → 이상치 수: 142 (범위: -337.73 ~ 1116.92)
매출_투자비 → 이상치 수: 205 (범위: -3.27 ~ 8.25)
팔로워_고객비 → 이상치 수: 131 (범위: -0.10 ~ 0.17)
이익률 → 이상치 수: 227 (범위: -0.99 ~ 1.79)
매출_성장률 → 이상치 수: 209 (범위: -2.92 ~ 3.52)
총_자본_비율 → 이상치 수: 227 (범위: -0.79 ~ 1.99)
직원당_매출 → 이상치 수: 195 (범위: -4.81 ~ 11.38)


In [920]:
# 클리핑
def clip_outliers_iqr(df, col):
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    df[col] = df[col].clip(lower, upper)

In [921]:
# 클리핑 적용
for col in ['기업가치_연차비', '매출_투자비', '팔로워_고객비']:
    clip_outliers_iqr(train, col)
for col in ['기업가치_연차비', '매출_투자비', '팔로워_고객비']:
    clip_outliers_iqr(test, col)

### 5. 인코딩
- 변수 : 국가, 분야, 투자단계, 인수여부, 상장여부
- One-Hot Encoding (카테고리가 적을 때)
- Label Encoding (카테고리가 많거나 순서가 있을 때)

In [922]:
# one-hot 인코딩을 쓸지 label 인코딩을 쓸지 카테고리 수 파악
# 확인할 컬럼 리스트
cat_cols = ['국가', '분야', '투자단계', '인수여부', '상장여부']

# 각 컬럼의 카테고리 수 출력
for col in cat_cols:
    n_unique = train[col].nunique(dropna=True)
    print(f"'{col}' 카테고리 수: {n_unique}")
    print(f"→ 카테고리 목록: {train[col].dropna().unique()}\n")

'국가' 카테고리 수: 10
→ 카테고리 목록: ['CT005' 'CT006' 'CT007' 'CT002' 'CT008' 'CT010' 'CT001' 'CT009' 'CT003'
 'CT004']

'분야' 카테고리 수: 11
→ 카테고리 목록: ['이커머스' '핀테크' '기술' '기타' '에듀테크' '게임' '헬스케어' '물류' '푸드테크' 'AI' '에너지']

'투자단계' 카테고리 수: 5
→ 카테고리 목록: ['Series A' 'Seed' 'Series C' 'Series B' 'IPO']

'인수여부' 카테고리 수: 2
→ 카테고리 목록: ['No' 'Yes']

'상장여부' 카테고리 수: 2
→ 카테고리 목록: ['No' 'Yes']



In [923]:
# 카테고리 수가 10개 이하이므로 one-hot 인코딩으로 선택
train = pd.get_dummies(train, columns=cat_cols, drop_first=False, dtype=int)
test = pd.get_dummies(test, columns=cat_cols, drop_first=False, dtype=int)
# 컬럼 불일치 일시
missing_cols = set(train.columns) - set(test.columns)
missing_cols.discard('성공확률')  # test에 없는 타겟컬럼은 제외
for col in missing_cols:
    test[col] = 0

### 6. 스케일링
- StandardScaler	평균 0, 표준편차 1로 변환. 정규분포에 가까운 경우 적합.
- MinMaxScaler	0~1 범위로 정규화. 데이터 분포에 관계없이 모든 범위 고정. 이상치에 민감함.
- RobustScaler	중앙값(median)과 IQR(4분위 범위) 기준으로 스케일링. 이상치에 강함.
- MaxAbsScaler	절댓값 기준으로 -1~1로 조정. 희소행렬(sparse matrix) 에 적합.

In [924]:
# 스케일링 방식을 비교해봤을 때 스케일링 대상 컬럼들 간의 이상치 영향이 크지 않거나, 선형 회귀 모델에 미치는 영향이 제한적이라 판단
# 이 경우 StandardScaler를 사용하는 것이 무난하다고 생각이 들어 StandardScaler 사용
from sklearn.preprocessing import StandardScaler

# 스케일링할 수치형 컬럼 리스트
numeric_cols = ['직원 수', '고객수(백만명)', '총 투자금(억원)', 
                '연매출(억원)', 'SNS 팔로워 수(백만명)', '기업가치(백억원)', 
                '설립연차', '기업가치_연차비', '매출_투자비', '팔로워_고객비', '이익률', '매출_성장률', '총_자본_비율', '직원당_매출']

# 스케일러 학습 및 변환
scaler = StandardScaler()
train[numeric_cols] = scaler.fit_transform(train[numeric_cols])

# test 데이터도 동일한 스케일러로 변환
test[numeric_cols] = scaler.transform(test[numeric_cols])

In [925]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4376 entries, 0 to 4375
Data columns (total 47 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              4376 non-null   object 
 1   설립연도            4376 non-null   int64  
 2   직원 수            4376 non-null   float64
 3   고객수(백만명)        4376 non-null   float64
 4   총 투자금(억원)       4376 non-null   float64
 5   연매출(억원)         4376 non-null   float64
 6   SNS 팔로워 수(백만명)  4376 non-null   float64
 7   기업가치(백억원)       4376 non-null   float64
 8   성공확률            4376 non-null   float64
 9   설립연차            4376 non-null   float64
 10  기업가치_연차비        4376 non-null   float64
 11  매출_투자비          4376 non-null   float64
 12  팔로워_고객비         4376 non-null   float64
 13  이익률             4376 non-null   float64
 14  매출_성장률          4376 non-null   float64
 15  총_자본_비율         4376 non-null   float64
 16  직원당_매출          4376 non-null   float64
 17  국가_CT001        4376 non-null   i

### 7. 모델링

In [926]:
# target과 제외할 칼럼
target_col = '성공확률'
drop_cols = ['ID', '총_자본_비율', target_col]

# 학습용 피처, 타겟 분리
X = train.drop(columns=drop_cols)
y = train[target_col]

In [927]:
from sklearn.model_selection import train_test_split
# 훈련/검증 데이터 분할
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

In [928]:
import numpy as np
import pandas as pd
from collections import Counter

def calculate_weighted_mae(y_true, y_pred):
    """
    Weighted MAE 계산 함수

    Parameters:
    - y_true: 실제값 (1D array 또는 Series)
    - y_pred: 예측값 (1D array 또는 Series)

    Returns:
    - weighted_mae: 가중 평균 절대 오차
    """
    # numpy array로 변환
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # 1. 정답값의 빈도 기반 가중치 계산
    freq = Counter(y_true)
    weights = np.array([1 / freq[val] for val in y_true])  # 빈도 낮을수록 가중치 ↑

    # 2. 가중 절대 오차 계산
    abs_errors = np.abs(y_true - y_pred)
    weighted_mae = np.sum(weights * abs_errors) / np.sum(weights)

    return weighted_mae

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error

# ▶ 모델 정의
rf_model = RandomForestRegressor(
    max_depth=None, 
    max_features='sqrt', 
    min_samples_leaf=2, 
    min_samples_split=5, 
    n_estimators=300,
    random_state=42
)
xgb_model = XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)

# ▶ 모델 학습
rf_model.fit(X_train, y_train)
xgb_model.fit(X_train, y_train)
lgbm_model.fit(X_train, y_train)

# ▶ 예측 (모든 모델)
rf_pred = rf_model.predict(X_valid)
xgb_pred = xgb_model.predict(X_valid)
lgbm_pred = lgbm_model.predict(X_valid)

# ▶ 앙상블 (단순 평균)
ensemble_pred = (0.2 * rf_pred + 0.4 * xgb_pred + 0.4 * lgbm_pred)

# ▶ 평가
mae = mean_absolute_error(y_valid, ensemble_pred)

print("앙상블 모델 MAE:", round(mae, 4))

# Weighted MAE
wmae = calculate_weighted_mae(y_valid, ensemble_pred)
print("Weighted MAE:", round(wmae, 4))

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000325 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2800
[LightGBM] [Info] Number of data points in the train set: 3500, number of used features: 44
[LightGBM] [Info] Start training from score 0.534486
앙상블 모델 MAE: 0.2013
Weighted MAE: 0.2086


In [ ]:
# 앙상블 모델 정의 (튜닝된 RandomForest 포함)
X_test = test.drop(columns=['ID', '총_자본_비율'])

rf_model = RandomForestRegressor(
    max_depth=None, 
    max_features='sqrt', 
    min_samples_leaf=2, 
    min_samples_split=5, 
    n_estimators=300,
    random_state=42
)
xgb_model = XGBRegressor(n_estimators=100, random_state=42, verbosity=0)
lgbm_model = LGBMRegressor(n_estimators=100, random_state=42)

# 전체 학습 데이터로 학습
rf_model.fit(X, y)
xgb_model.fit(X, y)
lgbm_model.fit(X, y)

# 테스트 데이터 예측
rf_pred_test = rf_model.predict(X_test)
xgb_pred_test = xgb_model.predict(X_test)
lgbm_pred_test = lgbm_model.predict(X_test)

# 앙상블 (가중 평균)
ensemble_pred_test = (0.2 * rf_pred_test + 0.4 * xgb_pred_test + 0.4 * lgbm_pred_test)

# sample_submission 불러오기
sample_submission = pd.read_csv('data/sample_submission.csv')
sample_submission['성공확률'] = ensemble_pred_test

# 저장
sample_submission.to_csv('data/ensemble_submission.csv', index=False, encoding='utf-8-sig')
print("✅ 앙상블 예측 결과가 저장되었습니다: ensemble_submission.csv")

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000211 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2803
[LightGBM] [Info] Number of data points in the train set: 4376, number of used features: 44
[LightGBM] [Info] Start training from score 0.537340
✅ 앙상블 예측 결과가 저장되었습니다: ensemble_submission.csv
